In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from workflow_widgets import WorkflowWidget, WorkerPoolWidget
from droplet_workflow import droplet_wf
from example_workflow import example_wf
import workflow_objects as kale
import ipywidgets as ipw

In [15]:
import traitlets as tr

# Example Workflow

In [17]:
WorkflowWidget(example_wf)

A Jupyter Widget

In [ ]:
wpw = WorkerPoolWidget()
wpw

In [ ]:
wpw.add_pool("Example Pool", 4)

In [ ]:
pool = wpw.get_pool("Example Pool")
pool.fw_run(example_wf)

---

# Droplet Workflow

In [ ]:
WorkflowWidget(droplet_wf)